# Neural Network using light sensor
This demonstrates a simple method where learning can be done using a neural network.

### Setup
We load the lego modules on the brick and connect sensors to the correct outputs.  
Note that we assume specific output ports!

In [1]:
import rpyc
import numpy as np
from IPython import display

conn = rpyc.classic.connect("ev3dev.local") # host name or IP address of the EV3
ev3 = conn.modules['ev3dev2']      # import ev3dev2.ev3 remotely

legosensors=conn.modules['ev3dev2.sensor.lego']
sensor_controls=conn.modules['ev3dev2.sensor']

sound=conn.modules['ev3dev2.sound']
sound=sound.Sound()
        

In [2]:
out1=sensor_controls.INPUT_1
out2=sensor_controls.INPUT_2
out3=sensor_controls.INPUT_3

touch = legosensors.TouchSensor(out3)
touchstop = legosensors.TouchSensor(out2)
color =legosensors.ColorSensor(out1)


# Tensorflow neural network
Tensorflow implementation.  
When both buttons are pushed, the system predicts based on input from the color detector. It then awaits a True/False answer (depends on button push), after which it trains with this as the response. 

In [13]:
Hidden_size = 10

In [4]:
import tensorflow as tf
color_names=["Black","Blue","Green","Red","White","Yellow"]
number_of_colors=len(color_names)
def wrong_color(color):
    return([(1-int(col==color)) for col in color_names])

def correct_color(color):
    return([int(col==color) for col in color_names])
    

C:\Users\Staris\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [29]:
data_X = []
data_y = []

In [30]:
tf.reset_default_graph()

# learning rate, this wildly affects response, and is high such that an early reaction is visible.
lr = 1  

# Placeholders and layers. The model has one hidden layer with sigmoid activation.
x = tf.placeholder(shape=[None, 5], dtype=tf.float32)
y = tf.placeholder(shape=[None, number_of_colors], dtype=tf.float32)

l = tf.layers.dense(x, Hidden_size, use_bias=True, activation=tf.nn.sigmoid)
output_layer = tf.layers.dense(l,number_of_colors,use_bias=False,activation=tf.nn.softmax)



# Classic softmax loss and gradient descent optimizer 
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output_layer, labels=y)

my_opt = tf.train.GradientDescentOptimizer(lr)
train_step = my_opt.minimize(loss)


with tf.Session() as sess:
    # Initialize variables
    init = tf.global_variables_initializer()
    sess.run(init)
    
    pred=False
    
    while True:   # Training loop. Instead of training over stored data, we respond to input.
        
        # both buttons pressed -> predict based on observation
        if touch.is_pressed and touchstop.is_pressed and pred==False:
            
            color_raw=[[ele/255 for ele in color.rgb]+[color.ambient_light_intensity/100]+[color.reflected_light_intensity/100]]
            
            output=sess.run(output_layer,feed_dict={x:color_raw})[0]
            
            
            display.clear_output(wait=True)
            for i in range(number_of_colors):
                display.display(str(color_names[i])+" : "+str(output[i]))
#                display.display(output)
            
            predict=color_names[np.argmax(output)]
            
            sound.speak(predict)
            
            pred=True
            
        # one button pressed -> train based on response
        
        if touch.is_pressed and not touchstop.is_pressed and pred==True:
            
            display.display("True")
            
            sess.run(train_step, feed_dict={x: color_raw, y: [correct_color(predict)]})
            
            data_X.append(color_raw)
            data_y.append(correct_color(predict))
            
            pred=False
                
        if touchstop.is_pressed and not touch.is_pressed and pred==True:
            
            display.display("False")
            
            sess.run(train_step, feed_dict={x: color_raw, y: [wrong_color(predict)]})

            data_X.append(color_raw)
            data_y.append(wrong_color(predict))
            
            pred=False

'Black : 0.13146167'

'Blue : 0.20897976'

'Green : 0.15094219'

'Red : 0.22930987'

'White : 0.14744666'

'Yellow : 0.13185982'

'False'

KeyboardInterrupt: 